# Impor library yang dibutuhkan

In [1]:
%pip install google_play_scraper
import os
import pandas as pd
from google_play_scraper import reviews, Sort
import time  # Untuk memberi jeda jika diperlukan
from tqdm import tqdm

Note: you may need to restart the kernel to use updated packages.


# --- Konfigurasi Scraping ---

In [2]:
# Daftar APP_ID target untuk masing-masing aplikasi:
LIST_APP_ID = [
    # E-commerce Umum / Marketplace
    'com.tokopedia.tkpd',             # Tokopedia
    'com.lazada.android',             # Lazada
    'com.shopee.id',                  # Shopee
    'blibli.mobile.commerce',         # Blibli
    'com.bukalapak.android',          # Bukalapak
    'bmd.android.apps',               # Bhinneka

    # === E-commerce Spesialis ===
    'com.zalora.android',             # Zalora (Fashion)
    'com.socialbella.soco',           # Sociolla (Beauty)
    'com.orami.parenting.android',    # Orami (Parenting & Kebutuhan Rumah)
    'com.infotech.mapclub',           # MAPCLUB (Mitra Adiperkasa Brands)
    'com.eraspace.app',               # Eraspace (Erafone & Erajaya group - Electronics)
    'com.mobileappruparupa',          # Ruparupa (Kawan Lama Group - ACE, Informa, etc. - Home & Living)
    'id.co.ikea.android',             # IKEA Indonesia (Home & Living)
    'com.gramedia.retail',            # Toko Gramedia   (Books & Stationery)
    'com.gramedia.digital',           # Gramedia Digital (Books & Stationery)

    # E-commerce Grocery / Convenience
    'com.indomaret.klikindomaret',    # Klik Indomaret
    'com.alfamart.alfagift',          # Alfagift (Alfamart)

    # === C2C / Second-hand Marketplace ===
    'com.thecarousell.Carousell',   # Carousell

    # === Travel E-commerce (OTA - Online Travel Agent) ===
    'com.traveloka.android',        # Traveloka (Tiket Pesawat, Hotel, Kereta, Aktivitas)
    'com.tiket.gits',               # Tiket.com (Tiket Pesawat, Hotel, Kereta, Event)

    # === Ticketing (Film/Event) ===
    'id.tix.android',               # TIX ID (Tiket Bioskop)

    # === Aplikasi Multi-layanan dengan Komponen E-commerce Kuat ===
    'com.grabtaxi.passenger',       # Grab (Termasuk GrabFood, GrabMart)

    # Aplikasi lain yang ada di daftar asli (non e-commerce, tapi dipertahankan sesuai request asli)
    'com.telkomsel.telkomselcm',            # MyTelkomsel (Utility/Provider)
    'co.pegadaian.syariah.nasabah',         # Pegadaian Syariah (Finance)
    'com.icon.pln123',                      # PLN Mobile (Utility)
    'com.gojek.partner',                    # GoPartner (Layanan - Driver) # Mengganti nama Gojek Driver
    'com.gojek.gopay',                      # GoPay (Finance/Payment)
    'com.shopeepay.id',                     # ShopeePay (Finance/Payment)
    'com.bnc.finance',                      # NeoBank / Bank Neo Commerce (Finance)

    # === Game & Hiburan dari daftar asli (jika masih relevan untuk perbandingan) ===
    # 'com.HoYoverse.Nap',                  # ZZZ (Game)
    # 'com.nexon.bluearchive',              # Blue Archive (Game)
    # 'com.robtopx.geometryjump',           # Geometry Dash (Game)
    # 'com.ea.game.pvzfree_row',            # Plants VS Zombie (Game)
    # 'com.gameloft.android.ANMP.GloftA8HM', # Asphalt 8 (Game)
    # 'com.miHoYo.bh3oversea',              # Honkai Impact 3 (Game)
    # 'com.HoYoverse.hkrpgoversea',         # Honkai: Star Rail (Game)
    # 'com.king.candycrushsaga',            # Candy Crush (Game)
    # 'com.bardi.smart.home',               # Bardi Smart Home (Utility/IoT)
    # 'com.linecorp.LGGRTHN',               # Line Get Rich (Game)
    # 'com.naver.linewebtoon',              # Webtoon (Entertainment)
    # 'com.netflix.mediaclient',            # Netflix (Entertainment)
    # 'com.discord',                        # Discord (Komunikasi)

]

JUMLAH_ULASAN_TARGET = 25000  # Target ulasan per aplikasi
NAMA_FILE_OUTPUT_SEMUA = 'hasil_scraping_semua_app.csv'

# Catatan tambahan:
# - ID 'ShopeePay' mungkin perlu diverifikasi ulang karena seringkali terintegrasi dalam aplikasi Shopee utama ('com.shopee.id').
#   Jika ada aplikasi ShopeePay terpisah yang ingin ditargetkan, pastikan ID-nya benar.
# - Beberapa aplikasi e-commerce lain mungkin ada (misalnya JD.ID yang sudah tidak beroperasi, atau platform yang lebih niche).
#   Daftar ini mencakup yang paling umum dan relevan saat ini.
# - Nama aplikasi Gojek Driver lebih umum dikenal sebagai GoPartner sekarang.

# Proses scraping untuk setiap aplikasi

In [3]:
# List untuk menampung DataFrame masing-masing aplikasi
list_df_app = []

for app in LIST_APP_ID:
    file_output_app = f"raw_app_data/hasil_scraping_{app}.csv"
    skip_download = False
    
    # Cek apakah file sudah ada
    if os.path.exists(file_output_app):
        print(f"\nFile {file_output_app} ditemukan. Memeriksa jumlah data...")
        df_existing = pd.read_csv(file_output_app, encoding='utf-8-sig')
        if len(df_existing) == JUMLAH_ULASAN_TARGET:
            print(f"Jumlah data pada {file_output_app} sudah memenuhi target ({JUMLAH_ULASAN_TARGET}). Skip download.")
            list_df_app.append(df_existing)
            skip_download = True
        else:
            print(f"Jumlah data pada {file_output_app} ({len(df_existing)}) tidak memenuhi target. Proses download ulang.")

    if not skip_download:
        print(f"\nMemulai scraping ulasan untuk aplikasi: {app}")
        print(f"Target jumlah ulasan: {JUMLAH_ULASAN_TARGET}")
        
        try:
            # Inisialisasi variabel untuk menyimpan ulasan dan token kelanjutan
            semua_ulasan = []
            token = None
            batch_size = 1000  # jumlah ulasan per iterasi
            
            # Variabel untuk mendeteksi ketika tidak ada ulasan baru
            previous_total = -1
            no_new_reviews_count = 0
            max_no_new_reviews = 3  # Berhenti setelah 3 kali tidak ada ulasan baru
            
            # Membuat progress bar dengan total target ulasan
            with tqdm(total=JUMLAH_ULASAN_TARGET, desc=f"Scraping {app}") as pbar:
                while True:
                    # Tambahkan kode retry: hingga 5 kali jika terjadi exception
                    for attempt in range(5):
                        try:
                            result, token = reviews(
                                app,
                                lang='id',           # Bahasa ulasan (Indonesia)
                                country='id',        # Negara (Indonesia)
                                sort=Sort.NEWEST,    # Urutkan berdasarkan terbaru
                                count=batch_size,
                                continuation_token=token
                            )
                            break  # Jika berhasil, keluar dari loop retry
                        except Exception as e:
                            print(f"\nGagal mengambil ulasan (attempt {attempt+1}/5) untuk {app}: {e}")
                            if attempt < 4:
                                print("Mencoba kembali...")
                                time.sleep(2)  # jeda selama 2 detik sebelum retry
                            else:
                                raise Exception(f"Gagal terhubung ke {app} setelah 5 kali percobaan") from e

                    # Periksa apakah ada ulasan baru yang diterima
                    if len(result) == 0:
                        print(f"\nTidak ada ulasan baru yang diterima dari {app}. Berhenti proses scraping.")
                        break
                        
                    # Tambahkan ulasan yang didapat ke list
                    semua_ulasan.extend(result)
                    
                    # Update progress bar hanya untuk ulasan baru
                    pbar.update(len(result))
                    
                    # Deteksi ketika jumlah ulasan tidak bertambah lagi
                    if previous_total == len(semua_ulasan):
                        no_new_reviews_count += 1
                        print(f"\nPeringatan: Tidak ada ulasan baru yang ditambahkan. Percobaan ke-{no_new_reviews_count}/{max_no_new_reviews}")
                        if no_new_reviews_count >= max_no_new_reviews:
                            print(f"\nSetelah {max_no_new_reviews} kali tidak ada ulasan baru, proses scraping dihentikan.")
                            print(f"Berhasil mengumpulkan {len(semua_ulasan)} ulasan dari {app}, yang mungkin adalah total ulasan yang tersedia.")
                            break
                    else:
                        # Reset counter jika ada ulasan baru
                        no_new_reviews_count = 0
                    
                    # Perbarui total sebelumnya
                    previous_total = len(semua_ulasan)
                    
                    # Jika sudah mencapai atau melebihi target atau tidak ada token lagi, berhenti
                    if token is None or len(semua_ulasan) >= JUMLAH_ULASAN_TARGET:
                        break
                    
                    # Tambahkan jeda kecil untuk menghindari rate limiting
                    time.sleep(0.5)

            # Potong jika ulasan yang didapat melebihi target
            if len(semua_ulasan) > JUMLAH_ULASAN_TARGET:
                print(f"Mengambil {len(semua_ulasan)} ulasan, memotong menjadi {JUMLAH_ULASAN_TARGET}.")
                semua_ulasan = semua_ulasan[:JUMLAH_ULASAN_TARGET]
            
            # Tambahkan pesan yang lebih informatif jika jumlah ulasan di bawah target
            if len(semua_ulasan) < JUMLAH_ULASAN_TARGET:
                print(f"\nCATATAN: Hanya berhasil mendapatkan {len(semua_ulasan)} ulasan dari {app}.")
                print(f"Kemungkinan aplikasi {app} memiliki lebih sedikit ulasan dari target ({JUMLAH_ULASAN_TARGET}).")
                print("Melanjutkan proses penyimpanan dengan jumlah ulasan yang tersedia.")

            print(f"Berhasil mendapatkan {len(semua_ulasan)} ulasan untuk aplikasi {app}.")

            # Tambahkan informasi app_id ke masing-masing ulasan untuk penanda
            for ulasan in semua_ulasan:
                ulasan['app_id'] = app

            # Konversi ke DataFrame dan simpan sebagai file CSV untuk aplikasi ini
            df_app = pd.DataFrame(semua_ulasan)
            kolom_yang_diinginkan = ['app_id', 'userName', 'score', 'at', 'content']
            df_app = df_app[kolom_yang_diinginkan]
            df_app.to_csv(file_output_app, index=False, encoding='utf-8-sig')
            print(f"Data ulasan untuk {app} berhasil disimpan ke file: {file_output_app}")

            # Simpan DataFrame ke list untuk penggabungan akhir
            list_df_app.append(df_app)

        except Exception as e:
            print(f"Terjadi kesalahan saat scraping untuk {app}: {e}")
            print("Pastikan ID Aplikasi benar dan koneksi internet stabil.")



File raw_app_data/hasil_scraping_com.tokopedia.tkpd.csv ditemukan. Memeriksa jumlah data...
Jumlah data pada raw_app_data/hasil_scraping_com.tokopedia.tkpd.csv (18000) tidak memenuhi target. Proses download ulang.

Memulai scraping ulasan untuk aplikasi: com.tokopedia.tkpd
Target jumlah ulasan: 25000


Scraping com.tokopedia.tkpd: 100%|██████████| 25000/25000 [00:54<00:00, 454.82it/s]


Berhasil mendapatkan 25000 ulasan untuk aplikasi com.tokopedia.tkpd.
Data ulasan untuk com.tokopedia.tkpd berhasil disimpan ke file: raw_app_data/hasil_scraping_com.tokopedia.tkpd.csv

File raw_app_data/hasil_scraping_com.lazada.android.csv ditemukan. Memeriksa jumlah data...
Jumlah data pada raw_app_data/hasil_scraping_com.lazada.android.csv (18000) tidak memenuhi target. Proses download ulang.

Memulai scraping ulasan untuk aplikasi: com.lazada.android
Target jumlah ulasan: 25000


Scraping com.lazada.android: 100%|██████████| 25000/25000 [00:52<00:00, 472.27it/s]


Berhasil mendapatkan 25000 ulasan untuk aplikasi com.lazada.android.
Data ulasan untuk com.lazada.android berhasil disimpan ke file: raw_app_data/hasil_scraping_com.lazada.android.csv

File raw_app_data/hasil_scraping_com.shopee.id.csv ditemukan. Memeriksa jumlah data...
Jumlah data pada raw_app_data/hasil_scraping_com.shopee.id.csv (18000) tidak memenuhi target. Proses download ulang.

Memulai scraping ulasan untuk aplikasi: com.shopee.id
Target jumlah ulasan: 25000


...

Berhasil mendapatkan 25000 ulasan untuk aplikasi com.gojek.gopay.
Data ulasan untuk com.gojek.gopay berhasil disimpan ke file: raw_app_data/hasil_scraping_com.gojek.gopay.csv

File raw_app_data/hasil_scraping_com.shopeepay.id.csv ditemukan. Memeriksa jumlah data...
Jumlah data pada raw_app_data/hasil_scraping_com.shopeepay.id.csv (18000) tidak memenuhi target. Proses download ulang.

Memulai scraping ulasan untuk aplikasi: com.shopeepay.id
Target jumlah ulasan: 25000


Scraping com.shopeepay.id: 100%|██████████| 25000/25000 [00:54<00:00, 457.84it/s]


Berhasil mendapatkan 25000 ulasan untuk aplikasi com.shopeepay.id.
Data ulasan untuk com.shopeepay.id berhasil disimpan ke file: raw_app_data/hasil_scraping_com.shopeepay.id.csv

File raw_app_data/hasil_scraping_com.bnc.finance.csv ditemukan. Memeriksa jumlah data...
Jumlah data pada raw_app_data/hasil_scraping_com.bnc.finance.csv (18000) tidak memenuhi target. Proses download ulang.

Memulai scraping ulasan untuk aplikasi: com.bnc.finance
Target jumlah ulasan: 25000


Scraping com.bnc.finance: 100%|██████████| 25000/25000 [00:52<00:00, 471.97it/s]

Berhasil mendapatkan 25000 ulasan untuk aplikasi com.bnc.finance.
Data ulasan untuk com.bnc.finance berhasil disimpan ke file: raw_app_data/hasil_scraping_com.bnc.finance.csv


# --- Gabungkan semua data aplikasi ke dalam satu file CSV ---

In [4]:
if list_df_app:
    df_semua_app = pd.concat(list_df_app, ignore_index=True)
    df_semua_app.to_csv(NAMA_FILE_OUTPUT_SEMUA, index=False, encoding='utf-8-sig')
    print(f"\nData seluruh ulasan dari semua aplikasi berhasil disimpan ke file: {NAMA_FILE_OUTPUT_SEMUA}")
else:
    print("Tidak ada data ulasan yang berhasil dikumpulkan.")


Data seluruh ulasan dari semua aplikasi berhasil disimpan ke file: hasil_scraping_semua_app.csv
